In [1]:
import pandas as pd 
import numpy as np
from optlang import Objective, Variable, Constraint, Model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.dates as mdates
from bokeh.plotting import figure, output_file, ColumnDataSource
from bokeh.io import show
from bokeh.models import HoverTool
from bokeh.layouts import column
from Generator import Thermal_Generator
from EPEX_Scrapping import epex_DAM

In [2]:
%matplotlib inline
sns.set(style='whitegrid', context='notebook')
plt.rcParams['figure.figsize'] = (18.0, 7.5)
plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize
plt.rc('figure', titlesize=15)  # fontsize of the figure title

# 1. Procedure for Solving a Thermal Generator Dispatch profile problem

The procedure for creating and solving the dispatch problem for a thermal generator with given data comprises of 6 steps : 

#### 1.1 Data Importation

All the data in used in this model is in €.

In [6]:
data = pd.read_csv('data/CCGT_UK.csv', index_col='time', parse_dates=True)

#### 1.2 Create `Thermal_Generator` Object 

The input for the object is the timeseries commodity data in `price_data`, which incudes : electricity, gas and carbon prices. The parameters of the generator, if not specified, will be that of a typical CCGT of $100$ MW nominal capacity.

In [7]:
gen = Thermal_Generator(Name='Demo',price=data)

#### 1.3 Create the optimization model 

Depending on the length of the time interval given in `price_data`, the initialization time for this model takes several seconds. For large problem (1 year time horizon and with minimum offline constraint activated), this can take up to 1~2 minutes.

In [8]:
gen.optimization_problem()

Building the problem - Please wait
Variables
Constraints
Objective
Capacity Factor Constraint Activated 
Object Creation Finished


#### 1.4 Solve the optimization problem  

The solving time depends on the length of the time horizon as well as whether the capacity factor and minimum offline constraints are activated or not. Note that if the solver status indicates `infeasible` or `unbounded`, then the solver returns the problem **excluding** the constraints or variables that make the problem `infeasible` or `unbounded`.

In [9]:
gen.solve_optim_problem()

Solving the problem - please wait
Solver Status : OPTIMAL
Objective Value :  7587631.295974148


#### 1.5 Get the solutions and relevent indicators 

More on this will be discussed in the result processing part.

In [10]:
output = gen.solution_values()

Gross Profit:  4.785329 € mil


#### 1.6 Visualize the dispatch profile 

In [11]:
gen.visualization_interactive(mode='Notebook')

Loading BokehJS ...

# 2. Data Acquisition and Model Creation

This part introduces in further detail the first three steps of the procedure above. 

### 2.1 Data Acquisition 

The timeseries data required for this model includes : Electricity, Fossil Fuel (Gas, Coal) Day-Ahead Markets' prices and Carbon price. The fossil fuel and carbon prices are not as fluctuating, neither influential on the dispatch profile as that of electricity. Hence, the electricity DAM price is of the greatest importance here. The function `epex_DAM` is provided in order to facilitate the electricity DAM data acquisition step by scrapping prices from [EPEX SPOT Day-ahead auction price](https://www.epexspot.com/en/market-data/dayaheadauction/auction-table/). 

We can scrap the data for three regions listed there, namely France (code `FR`), German (code `DE/AT`) or Czech Republic (code `CH`) dating back from a chosen `start` date with an interval with the unit of weeks `nbr_weeks`. The function returns two DataFrames of different structure, but containing the same data.

In [12]:
start_date = pd.Timestamp.today().date()
df, df_hour = epex_DAM(start=start_date, country='DE/AT', nbr_weeks=1)

In [13]:
df.head()

,00,01,02,03,04,05,06,07,08,09,...,14,15,16,17,18,19,20,21,22,23
2018-08-12,48.22,45.00,42.80,42.05,40.99,40.93,41.47,40.97,41.10,39.90,...,21.96,24.83,27.33,42.56,51.19,57.55,61.35,63.02,62.10,57.73
2018-08-11,49.43,45.80,44.02,41.39,40.64,41.01,41.52,44.22,47.97,47.63,...,23.44,28.19,33.89,41.09,49.60,57.98,58.92,60.53,60.68,58.39
2018-08-10,32.68,24.40,18.68,18.43,18.49,29.40,43.81,49.69,52.02,52.77,...,44.05,44.70,48.04,52.96,59.36,61.95,61.50,58.99,57.00,53.79
2018-08-09,50.57,47.47,46.14,45.07,45.08,47.76,55.50,64.11,65.08,64.88,...,57.56,56.03,54.39,56.54,58.76,56.90,52.36,50.00,49.54,45.78
2018-08-08,47.77,46.41,45.49,43.90,43.90,46.66,51.15,58.71,65.06,65.04,...,49.44,50.97,54.56,62.96,65.70,67.85,67.68,68.00,68.01,64.97


In [14]:
df_hour.tail()

,DAM
Time,
2018-08-12 19:00:00,57.55
2018-08-12 20:00:00,61.35
2018-08-12 21:00:00,63.02
2018-08-12 22:00:00,62.10
2018-08-12 23:00:00,57.73


# 3. Results Processing 

### 3.1 Operational Profile: 

Further explorations of the operational profile for the given time horizon can be found with the `Operation_Profile` attribute, which is a dictionary containing different keys:

**`STMC`** : Short Term Marginal Cost. This is the main criterion  to decide the operational modes for the generator (Startup, Shutdown, Generation modes, which are specified in the input lists : `Efficiency` and `Power`). The `STMC` is the aggregation of three costs : 
- Fuel cost ($€/MWh_{e}$)
- Carbon cost ($€/MWh_{e}$)
- Variable O&M cost ($€/MWh_{e}$)

The full formula for `STMC` will be given in the mathematical formulation document  

**`Capacity Factor` **: Capacity factor, which is the total number of operational hours of the horizon over the entire year (8670 hours).

\begin{align}
    CF = \frac{\sum_{t}^{N_{time}} \sum_{m}^{N_{mode}} X_{t}^{m}}{8760}
\end{align}

where $X_{t}^{m}$ represents the state of the operation mode `m` of the generator at time $t$ (0 means the operator is not operating in that mode, 1 otherwise)

**`Start-up Numbers`** : the number of startups within the given time horizon

**`Energy_Profile`** : the dispatch profile of the generator, given in a form of a `DataFrame`. It is the `Generation` line plot in the visualization above. The columns' labels represent the energy produced in different operation modes

In [15]:
gen.Operation_Profile['Energy_Profile'].tail(10)

,0,1,Total
time,,,
2017-12-31 14:00:00,0,0,0
2017-12-31 15:00:00,0,0,0
2017-12-31 16:00:00,0,0,0
2017-12-31 17:00:00,0,0,0
2017-12-31 18:00:00,0,0,0
2017-12-31 19:00:00,0,0,0
2017-12-31 20:00:00,0,0,0
2017-12-31 21:00:00,0,0,0
2017-12-31 22:00:00,0,0,0


### 3.2 Financial Metrics : 

We can explore different economic indicators of the generator for the given time horizon with the  `Finance_Metrics` attribute, which is also a dictionary containing different items: 

**`OPEX`**: this is the aggregate operation expenditure that occurs within the horizon, including : fuel cost, carbon cost, variable and fixed O&M costs, startups cost.

**`Average STMC`** : mean STMC 

**`Gross Profit`** : profit before tax

**`Revenue`** : revenue coming from electricity trading in the EPEX Day-ahead market

In [16]:
for keys, vals in gen.Finance_Metrics.items():
    print(keys, ':', vals)

Revenue : 70771870.2
OPEX : 65986540.27388887
Gross Profit : 4785329.926111135
Average STMC : 44.59493281604857


### 3.3 Optimization Model and solutions

The main model for the generator is stored in the `.optim_model` attributes. After the problem has been solved using `solve_optim_problem` method, the solutions (`solutions` attribute) as well as the corresponding metrics and indicators are acquired through the `solution_values` method. 

In [19]:
gen.solutions.head(15)

,Shut,Start,state_mode_0,state_mode_1
time,,,,
2017-01-01 00:00:00,0,0,0,0
2017-01-01 01:00:00,0,0,0,0
2017-01-01 02:00:00,0,0,0,0
2017-01-01 03:00:00,0,0,0,0
2017-01-01 04:00:00,0,0,0,0
2017-01-01 05:00:00,0,0,0,0
2017-01-01 06:00:00,0,0,0,0
2017-01-01 07:00:00,0,0,0,0
2017-01-01 08:00:00,0,1,0,0


In the solutions above, we can see that the generator is in turned on at 2015-08-01 03:00:00, and by default, the minimum number of hours before generation after startup is 2 hours, that's why the generator begin the operation in the `mode_0` at 2015-08-01 06:00:00.   	

### 3.4 Visualization : 

The dispatch profile and price/cost variation can be visualized using two available methods : 

**`visualization_non_interactive`** : non-interactive visualization with `seaborn` and `matplotlib`

**`visualization_interactive`** : interactive visualization with `bokeh`.



In [18]:
gen.visualization_interactive(mode='Notebook')

Loading BokehJS ...

We have gone through the necessary procedure for solving a problem of the dispatch profile for a thermal generator. The validation of this model can be found through the analysis of the carbon price mechanisms impact on thermal generators in the UK and Germany, which can be found in the `Case Study - Carbon Price Mechanism` notebook. 